<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold">
<br>RUNNING FANAL ANALYSIS</p><br>

This notebook illustrates how to run the FANAL analysys departing from NEXUS files.

The result of the FANAL analysis will be the probability of the simulated events to be interpreted as a signal event so, although we typycally refer to it as "*rejection factor*", strictly speaking it is an "*acceptance factor*". It also provides in a Pandas DataFrame format all the information related to "events", "tracks" and "vovels" generated during the analysis.

The process to run the analysis consists in creating a FanalSetup instance with its details (detector, input nexus files, output fils, analysis parameters ...), and the action of running the analysis. This NB illustrates 2 different ways to build the setup and run the analysis:
1. Loading the Setup config parameters from a config file (json formatted).
2. Building the Setup by providing by hand all the parameters needed.

`FANAL analysis` process is run in a per-event basis and basically consists on a series of filters that events must fulfill to pass to the next analysis step. Events passing all the filters are considered as signal events. Following, a summary of the main steps of the analysis is presented:
> * `MC filter`: Checks that MC event energy fits into limits, there is no signal in BUFFER, and there event has a single S1 signal.
> * Reconstructed hits = MC ionization hits with energy and positions smeared; and shifted z_position (if needed by hit time).
> * `energy filter`: Checks if the smeared event energy fits into limits.
> * Voxelize reconstructed hits using IC Paolina package.
> * `fiducial filter`: Checks if there is any voxel in the non-fiducial region with energy higher than threshold.
> * Make tracks from voxels using IC Paolina package.
> * `track filter`: Checks if the number of tracks is lower or equal the number set by parameter. Currently set to 1.
> * Get blobs from tracks using IC Paolina package.
> * `blob filter`: Checks if blob energies are higher than a certain energy threshold set by parameter.
> * `ROY filter`: Checks if the event smeared energy fts into ROI limits set by parameter.

The NB "fanal_results.ipynb" shows the different data stored in FANAL analysis with associated plots.

#### Setting general stuff

In [1]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib inline

In [4]:
%load_ext memory_profiler
%load_ext line_profiler

import line_profiler
profile = line_profiler.LineProfiler()

#### Importing

In [5]:
# General importings

import glob
import json

In [6]:
# Specific IC stuff

import invisible_cities.core.system_of_units  as units

In [8]:
# Specific FANAL stuff

from fanal.fanal_setup       import Setup
from fanal.core.fanal_types  import AnalysisParams

# RUNNING FANAL

## from a given config file (json format) ...

In [9]:
config_fnames = glob.glob("../config/*.config")
print ('Available config files:\n')
for config_fname in config_fnames: print(config_fname) 

Available config files:

../config/fanal.next100.bb0nu.fwhm_07.voxel_10x10x10.config
../config/fanal.next100.Bi214.fwhm_05.voxel_3x3x3.config
../config/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.config
../config/fanal.next100.Tl208.fwhm_07.voxel_10x10x10.config
../config/fanal.next100.Bi214.fwhm_07.voxel_10x10x10.config
../config/fanal.next100.Tl208.fwhm_05.voxel_3x3x3.config


In [12]:
#config_fname = "../config/fanal.next100.Bi214.fwhm_05.voxel_3x3x3.config"
config_fname = "../config/fanal.next100.Bi214.fwhm_07.voxel_10x10x10.config"
#config_fname = "../config/fanal.next100.Tl208.fwhm_05.voxel_3x3x3.config"
#config_fname = "../config/fanal.next100.Tl208.fwhm_07.voxel_10x10x10.config"
#config_fname = "../config/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.config"
#config_fname = "../config/fanal.next100.bb0nu.fwhm_07.voxel_10x10x10.config"

fanal_setup = Setup.from_config_file(config_fname)

In [13]:
%%time

fanal_setup.run_analysis()

*******************************************************************************
*** Detector:          NEXT100
*** Reconstructing:    Bi214 events
*** Input  files:      /Users/Javi/Development/FANAL/data/next100/Bi214/sim/*.h5  (5 files)
*** Output file:       /Users/Javi/Development/FANAL/data/next100/Bi214/fanal/fanal.next100.Bi214.fwhm_07.voxel_10x10x10.h5
*** Buffer energy th.: 2.0 keV
*** Transverse   diff: 0.00  mm/cm**0.5
*** Longitudinal diff: 0.00  mm/cm**0.5
*** Energy Resolution: 0.70% fwhm at Qbb
*** Voxel Size:        (10.0, 10.0, 10.0) mm  -  strict: False
*** Voxel energy th.:  2.0 keV
*** Track energy th.:  3.0 keV
*** Max num Tracks:    1
*** Blob radius:       18.0 mm
*** Blob energy th.:   350.0 keV
*** ROI energy limits: (2449.0, 2466.0) keV
*******************************************************************************


*** Processing /Users/Javi/Development/FANAL/data/next100/Bi214/sim/next100.Bi214.000.next.h5  (196 events) ...

* Num analyzed events: 1
* Num a

## from parameters ...

In [14]:
# General stuff
det_name        = 'NEXT100'
event_type      = 'bb0nu'
input_fname     = '../data/next100/bb0nu/sim/*.h5'
output_fname    = '../data/next100/bb0nu/fanal/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.h5'
verbosity_level = 'WARNING' # ('DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL')

analysis_params = AnalysisParams(buffer_Eth        = 0.   * units.keV,
                                 trans_diff        = 0.0  * units.mm / units.cm**0.5,
                                 long_diff         = 0.0  * units.mm / units.cm**0.5,
                                 fwhm              = 0.7  * units.perCent,
                                 e_min             = 2.4  * units.MeV,
                                 e_max             = 2.5  * units.MeV,
                                 voxel_size_x      = 10.  * units.mm,
                                 voxel_size_y      = 10.  * units.mm,
                                 voxel_size_z      = 10.  * units.mm,
                                 strict_voxel_size = True,
                                 voxel_Eth         = 0.   * units.keV,
                                 veto_width        = 20.  * units.mm,
                                 veto_Eth          = 10.  * units.keV,
                                 track_Eth         = 0.   * units.keV,
                                 max_num_tracks    = 1,
                                 blob_radius       = 21.  * units.mm,
                                 blob_Eth          = 350  * units.keV,
                                 roi_Emin          = 2454 * units.keV,
                                 roi_Emax          = 2471 * units.keV
                                )


# Executing FANAL
fanal_setup = Setup(det_name        = det_name,
                    event_type      = event_type,
                    input_fname     = input_fname,
                    output_fname    = output_fname,
                    analysis_params = analysis_params,
                    verbosity       = verbosity_level)

In [15]:
%%time

fanal_setup.run_analysis()

*******************************************************************************
*** Detector:          NEXT100
*** Reconstructing:    bb0nu events
*** Input  files:      ../data/next100/bb0nu/sim/*.h5  (5 files)
*** Output file:       ../data/next100/bb0nu/fanal/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.h5
*** Buffer energy th.: 0.0 keV
*** Transverse   diff: 0.00  mm/cm**0.5
*** Longitudinal diff: 0.00  mm/cm**0.5
*** Energy Resolution: 0.70% fwhm at Qbb
*** Voxel Size:        (10.0, 10.0, 10.0) mm  -  strict: True
*** Voxel energy th.:  0.0 keV
*** Track energy th.:  0.0 keV
*** Max num Tracks:    1
*** Blob radius:       21.0 mm
*** Blob energy th.:   350.0 keV
*** ROI energy limits: (2454.0, 2471.0) keV
*******************************************************************************


*** Processing ../data/next100/bb0nu/sim/next100.bb0nu.000.next.h5  (100 events) ...

* Num analyzed events: 1
* Num analyzed events: 2
* Num analyzed events: 3
* Num analyzed events: 4
* Num analyzed ev

# SHOWING FANAL RESULTS

In [16]:
config_df  = fanal_setup.config_df()
events_df  = fanal_setup.events_df()
tracks_df  = fanal_setup.tracks_df()
voxels_df  = fanal_setup.voxels_df()
results_df = fanal_setup.results_df()

In [17]:
events_df.columns

Index(['num_mcParts', 'num_mcHits', 'mc_energy', 'mc_filter', 'sm_energy',
       'energy_filter', 'num_voxels', 'voxel_size_x', 'voxel_size_y',
       'voxel_size_z', 'veto_energy', 'fiduc_filter', 'num_tracks',
       'track_length', 'track_filter', 'blob1_energy', 'blob2_energy',
       'blob_filter', 'roi_filter'],
      dtype='object')

In [18]:
events_df[['num_mcParts', 'num_mcHits', 'mc_energy', 'sm_energy', 'energy_filter', 'num_voxels',
           'fiduc_filter', 'num_tracks', 'track_length', 'track_filter',
           'blob1_energy', 'blob2_energy', 'blob_filter', 'roi_filter']].head()

,num_mcParts,num_mcHits,mc_energy,sm_energy,energy_filter,num_voxels,fiduc_filter,num_tracks,track_length,track_filter,blob1_energy,blob2_energy,blob_filter,roi_filter
event_id,,,,,,,,,,,,,,
0,25,310,2.457830,2.453856,True,18,True,1,82.925287,True,1.021097,0.836346,True,False
1,22,310,2.457830,2.456342,True,25,True,1,107.067423,True,0.710483,0.643837,True,True
2,25,146,1.020527,NaN,False,-1,False,-1,NaN,False,NaN,NaN,False,False
3,22,330,2.328514,NaN,False,-1,False,-1,NaN,False,NaN,NaN,False,False
4,30,337,2.457830,2.449920,True,23,True,3,NaN,False,NaN,NaN,False,False


In [19]:
tracks_df.head()

energy      length  num_voxels     blob1_x     blob1_y  \
event_id track_id                                                             
0        0         2.453856   82.925287          18  334.422009 -293.837105   
1        0         2.456342  107.067423          25  180.922456  336.753514   
4        0         2.205143   97.067423          20         NaN         NaN   
         1         0.145430    0.000000           1         NaN         NaN   
         2         0.099347   10.000000           2         NaN         NaN   

                      blob1_z  blob1_energy  blob1_num_hits     blob2_x  \
event_id track_id                                                         
0        0         158.979311      1.021097             134  314.678711   
1        0         756.130707      0.710483              83  141.365537   
4        0                NaN           NaN               0         NaN   
         1                NaN           NaN               0         NaN   
         2                NaN           NaN               0         NaN   

                      blob2_y     blob2_z  blob2_energy  blob2_num_hits  \
event_id track_id                                                         
0        0        -312.846713  219.475003      0.836346             100   
1        0         358.011639  783.276210      0.643837              93   
4        0                NaN         NaN           NaN               0   
         1                NaN         NaN           NaN               0   
         2                NaN         NaN           NaN               0   

                   ovlp_energy  
event_id track_id               
0        0                 0.0  
1        0                 0.0  
4        0                 NaN  
         1                 NaN  
         2                 NaN

In [20]:
voxels_df.head()

x          y           z    energy
event_id track_id voxel_id                                             
0        0        0         314.150543 -316.16713  197.001541  0.038360
                  1         314.150543 -316.16713  207.001541  0.029124
                  2         314.150543 -316.16713  217.001541  0.367572
                  3         314.150543 -306.16713  197.001541  0.054326
                  4         314.150543 -306.16713  207.001541  0.075030

In [21]:
results_df

,events
simulated,500
stored,500
analyzed,500
mc_filter,357
energy_filter,357
fiduc_filter,313
track_filter,231
blob_filter,194
roi_filter,128


In [22]:
from fanal.containers.events import EventCounter

print(EventCounter(**results_df.events))

* Event counters ...
  Simulated    :        500  (1.00e+00)
  Stored       :        500  (1.00e+00)
  Analyzed     :        500  (1.00e+00)
  MC     filter:        357  (7.14e-01)
  Energy filter:        357  (7.14e-01)
  Fiduc. filter:        313  (6.26e-01)
  Track  filter:        231  (4.62e-01)
  Blob   filter:        194  (3.88e-01)
  ROI    filter:        128  (2.56e-01)

